# Understanding CaloCalTopoClusters vs CalTopoClusters

The training needs the calorimeter clusters as input - this is probably the most important input into the training algorithm. Hence, we should at least be conscious of any change we are going to make to those inputs.

Main points of discussion below:

* point 1

## Run 2

In run 2 we can find the training cluster code extractions in [DiVertAnalysis](https://gitlab.cern.ch/atlas-phys-exotics-llp-mscrid/fullrun2analysis/DiVertAnalysisR21) [here](https://gitlab.cern.ch/atlas-phys-exotics-llp-mscrid/fullrun2analysis/DiVertAnalysisR21/-/blob/master/DiVertAnalysis/Root/RegionVarCalculator_calRatio.cxx#L384):

```c++
...
int jetIndex = 0;
for (const auto &jet : *jets_nominal)
  {
    // CalRatio LLP NN variables
    xAOD::JetConstituentVector constits = jet->getConstituents();
    addToVectorBranch(vars, "nn_jet_index", jetIndex);

    // Arrays for fdeep
    double clus_pt_arr[30] = {};
    ...

    // Need counter to stop after 30 constituents
    int constit_counter = 0;

    // add constituents to ntuple, and also for fdeep arrays
    for (const auto &c : constits)
    {

      const xAOD::IParticle *rawObj = c->rawConstituent();
      const xAOD::CaloCluster *cl = dynamic_cast<const xAOD::CaloCluster *>(rawObj);

      addToVectorBranch(vars, "cluster_jetIndex", jetIndex);

      addToVectorBranch(vars, "clus_pt", c->pt());
      if (constit_counter < 30)
        clus_pt_arr[constit_counter] = c->pt();
      ...
    }

    ...

    if (isCleanJet_LooseBadLLP && (jet->pt() > 40000) && (fabs(jet->eta()) < 2.5))
    {
      // pre-process arrays in CalRatioUtils to match training pre-processing in python
      CalRatioUtils::getNNoutput(clus_pt_arr, clus_eta_arr, clus_phi_arr, clus_l1ecal_arr, clus_l2ecal_arr,
                                 clus_l3ecal_arr, clus_l4ecal_arr, clus_l1hcal_arr, clus_l2hcal_arr, clus_l3hcal_arr, clus_l4hcal_arr,
                                 clus_time_arr, track_pt_arr, track_eta_arr, track_phi_arr, track_vertex_nParticles_arr, track_chiSquared_arr,
                                 track_d0_arr, track_z0_arr, track_pixShared_arr, track_sctShared_arr, track_pixHoles_arr, track_sctHoles_arr,
                                 track_pixHits_arr, track_sctHits_arr, jet->eta(), jet->phi(), jet->pt(), mseg_etaPos_arr, mseg_phiPos_arr,
                                 mseg_etaDir_arr, mseg_phiDir_arr, mseg_t0_arr, mseg_chiSquared_arr, inputs, input_sequences,
                                 &clusters, &tracks, &msegs, &jets, false);

      ...
    }

    jetIndex++;
  }
```

Note a few things:

* We loop over `jet->getConstituents()` to get at the constituents.
* We are expecting these are the calo clusters that were used in jet reconstruction.
* I think they are pointing at the CaloClusters in the file.